# Import essential libraries

In [12]:
# Import joblib to save and load Python objects
import joblib
import os

# Import essential libraries
import pandas as pd
import numpy as np
import sqlite3

# ML preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ML models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# ML evaluation
from sklearn.metrics import accuracy_score
print("Libs importées avec succès.")

Libs importées avec succès.


# SQL treatement

In [13]:
# --- 2. CONNEXION À LA BASE DE DONNÉES ---
db_path = "../database/ELECTIONS.db"
conn = sqlite3.connect(db_path)
print(f"Connecté à la base de données : {db_path}")

Connecté à la base de données : ../database/ELECTIONS.db


Let's agregate our datas per mandants (2017-2022)

In [14]:
# --- 3. CHARGEMENT ET REMPLISSAGE INTELLIGENT DES DONNÉES (FORWARD-FILL) ---
query = """
SELECT
    DEPARTMENT_CODE,
    YEAR,
    WINNER,
    ROUND(AVG(POVERTY_RATE), 2) as avg_poverty_rate,
    ROUND(AVG(UNEMPLOYMENT_RATE), 2) as avg_unemployment_rate,
    ROUND(AVG(IMMIGRATION_RATE), 2) as avg_immigration_rate,
    ROUND(AVG(NUMBER_OF_VICTIMS), 0) as avg_number_of_victims
FROM ELECTIONS_ALL
GROUP BY DEPARTMENT_CODE, YEAR
ORDER BY DEPARTMENT_CODE, YEAR
"""
df_full = pd.read_sql(query, conn)

# Propager le dernier gagnant connu pour chaque département
df_full['WINNER'] = df_full.groupby('DEPARTMENT_CODE')['WINNER'].transform(lambda x: x.ffill())

# Supprimer les lignes qui restent sans gagnant (si les toutes premières années n'ont pas de valeur)
df_full.dropna(subset=['WINNER'], inplace=True)
print("Données chargées et complétées par forward-fill.")
print(f"Shape total du DataFrame après remplissage : {df_full.shape}")

Données chargées et complétées par forward-fill.
Shape total du DataFrame après remplissage : (752, 7)


Now that we have our training datas (first and 2nd mandats)  
We have to define our features (X) & target (Y)

In [15]:
# --- 4. DIVISION TEMPORELLE STRICTE (TRAIN < 2024, TEST = 2024) ---
train_df = df_full[df_full['YEAR'] < 2024].copy()
test_df = df_full[df_full['YEAR'] == 2024].copy()

# Par sécurité, on retire 'E.GAUCHE' qui a très peu d'échantillons et peut nuire à l'apprentissage
train_df = train_df[train_df['WINNER'] != 'E.GAUCHE']

print(f"\nTaille du jeu d'entraînement (2017-2023) : {train_df.shape}")
print(f"Taille du jeu de test (2024) : {test_df.shape}")


Taille du jeu d'entraînement (2017-2023) : (652, 7)
Taille du jeu de test (2024) : (94, 7)


## Features & Target

In [16]:
# --- 5. DÉFINITION DES FEATURES (X) ET DE LA CIBLE (y) ---
# Les features sont toutes les colonnes sauf 'WINNER' et 'YEAR'
X_train = train_df.drop(columns=['WINNER', 'YEAR'])
y_train = train_df['WINNER']
X_test = test_df.drop(columns=['WINNER', 'YEAR'])
y_test = test_df['WINNER']

print(f"\nJeux de données X_train/y_train et X_test/y_test créés.")


Jeux de données X_train/y_train et X_test/y_test créés.


Désormais, nous devons identifier les features numériques & catégorielles pour les différencier

In [17]:
# --- 6. CRÉATION DU PIPELINE DE PRÉTRAITEMENT ---
# Identification des colonnes
categorical_features = ['DEPARTMENT_CODE']
numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()

# Création du preprocessor pour les features (X)
preprocessor_X = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Création de l'encodeur pour la cible (y)
label_encoder_y = LabelEncoder()
print("\nPipeline de pré-traitement et encodeur de label créés.")


Pipeline de pré-traitement et encodeur de label créés.


In [18]:
# --- 7. APPLICATION DES TRANSFORMATIONS ---
# On "fit" (apprend) sur le jeu d'entraînement et on transforme les deux jeux
X_train_processed = preprocessor_X.fit_transform(X_train)
X_test_processed = preprocessor_X.transform(X_test)

y_train_encoded = label_encoder_y.fit_transform(y_train)
y_test_encoded = label_encoder_y.transform(y_test)
print("\nTransformations appliquées aux données d'entraînement et de test.")
print(f"Shape de X_train_processed: {X_train_processed.shape}")
print(f"Shape de X_test_processed: {X_test_processed.shape}")


Transformations appliquées aux données d'entraînement et de test.
Shape de X_train_processed: (652, 98)
Shape de X_test_processed: (94, 98)


## Export datas

In [20]:
# --- 8. PRÉPARATION DES DONNÉES FINALES POUR LA SAUVEGARDE ---
# Récupération des noms des colonnes après one-hot encoding
try:
    feature_names = preprocessor_X.get_feature_names_out()
except AttributeError: # Fallback pour les anciennes versions de scikit-learn
    ohe_feature_names = preprocessor_X.named_transformers_['cat']['onehot'].get_feature_names(input_features=categorical_features)
    feature_names = numerical_features + list(ohe_feature_names)

In [21]:
# Création des DataFrames finaux
X_train_processed_df = pd.DataFrame(X_train_processed.toarray() if hasattr(X_train_processed, "toarray") else X_train_processed, columns=feature_names)
X_test_processed_df = pd.DataFrame(X_test_processed.toarray() if hasattr(X_test_processed, "toarray") else X_test_processed, columns=feature_names)

# Ajout de la cible encodée pour la sauvegarde
train_to_db = X_train_processed_df.copy()
train_to_db['WINNER_encoded'] = y_train_encoded

test_to_db = X_test_processed_df.copy()
test_to_db['WINNER_encoded'] = y_test_encoded
print("\nDataFrames finaux prêts pour la sauvegarde.")


DataFrames finaux prêts pour la sauvegarde.


In [22]:
# --- 9. SAUVEGARDE DANS LA BASE DE DONNÉES ET DES TRANSFORMATEURS ---
# Sauvegarde des DataFrames dans SQLite
train_to_db.to_sql('PROCESSED_TRAIN_DATA', conn, if_exists='replace', index=False)
test_to_db.to_sql('PROCESSED_TEST_DATA', conn, if_exists='replace', index=False)
print(f"\nDonnées sauvegardées dans les tables 'PROCESSED_TRAIN_DATA' ({train_to_db.shape}) et 'PROCESSED_TEST_DATA' ({test_to_db.shape}).")


Données sauvegardées dans les tables 'PROCESSED_TRAIN_DATA' ((652, 99)) et 'PROCESSED_TEST_DATA' ((94, 99)).


In [24]:
# Sauvegarde des transformateurs en fichiers .joblib
db_dir = os.path.dirname(db_path)
preprocessor_path = os.path.join(db_dir, 'preprocessor_X.joblib')
label_encoder_path = os.path.join(db_dir, 'label_encoder_y.joblib')

joblib.dump(preprocessor_X, preprocessor_path)
joblib.dump(label_encoder_y, label_encoder_path)
print(f"Préprocesseur sauvegardé dans : {preprocessor_path}")
print(f"Encodeur de label sauvegardé dans : {label_encoder_path}")

Préprocesseur sauvegardé dans : ../database/preprocessor_X.joblib
Encodeur de label sauvegardé dans : ../database/label_encoder_y.joblib


In [25]:
# --- 10. FERMETURE DE LA CONNEXION ---
conn.close()
print("\nConnexion à la base de données fermée. Script terminé.")


Connexion à la base de données fermée. Script terminé.
